In [2]:
import requests 
import json
import os
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
from pandas import json_normalize
import numpy as np

In [2]:
keylast = os.getenv("keylast")

In [3]:
l_user = 'sinatxester'

# Conectar api lastfm: info usuarios

In [4]:
url_user = f'http://ws.audioscrobbler.com/2.0/?method=user.getinfo&user={l_user}&api_key={keylast}&format=json'

In [5]:
req_user = requests.get(url_user).json()['user']

In [6]:
req_user

{'country': 'Spain',
 'age': '0',
 'playcount': '77557',
 'subscriber': '0',
 'realname': 'Ester Sinatxe',
 'playlists': '0',
 'bootstrap': '0',
 'image': [{'size': 'small',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/daa41927775a61b3e4762c4a8eda80f3.png'},
  {'size': 'medium',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/daa41927775a61b3e4762c4a8eda80f3.png'},
  {'size': 'large',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/daa41927775a61b3e4762c4a8eda80f3.png'},
  {'size': 'extralarge',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/daa41927775a61b3e4762c4a8eda80f3.png'}],
 'registered': {'unixtime': '1420141064', '#text': 1420141064},
 'url': 'https://www.last.fm/user/sinatxester',
 'gender': 'n',
 'name': 'sinatxester',
 'type': 'user'}

In [7]:
req_user.keys()
#nos quedaremos con country, playcount,realname, image[1] (medium), registered['unixtime'],url,gender, name, type
#crearemos tabla en la base de datos de mysql con estos datos

dict_keys(['country', 'age', 'playcount', 'subscriber', 'realname', 'playlists', 'bootstrap', 'image', 'registered', 'url', 'gender', 'name', 'type'])

### Friends del user para conseguir mas amigos

In [8]:
url_fri = f'http://ws.audioscrobbler.com/2.0/?method=user.getfriends&user={l_user}&api_key={keylast}&format=json'

In [9]:
req_fri = requests.get(url_fri).json()['friends']

In [10]:
req_fri.keys()

dict_keys(['@attr', 'user'])

In [11]:
req_fri.keys()


dict_keys(['@attr', 'user'])

In [12]:
req_fri['@attr']

{'perPage': '50',
 'totalPages': '1',
 'page': '1',
 'total': '10',
 'user': 'sinatxester'}

In [13]:
req_fri['user'][0] #lista con todos los amiguismos

{'name': 'Lumanet',
 'url': 'https://www.last.fm/user/Lumanet',
 'country': 'Spain',
 'playlists': '0',
 'playcount': '0',
 'image': [{'size': 'small',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/25602223397211f1f083955ebeacf803.png'},
  {'size': 'medium',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/25602223397211f1f083955ebeacf803.png'},
  {'size': 'large',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/25602223397211f1f083955ebeacf803.png'},
  {'size': 'extralarge',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/25602223397211f1f083955ebeacf803.png'}],
 'registered': {'unixtime': '1196844580', '#text': '2007-12-05 08:49'},
 'realname': 'Marcos Robles',
 'subscriber': '0',
 'bootstrap': '0',
 'type': 'user'}

In [14]:
#para crear una tabla con los amigos, el nombre del usuario mismo es la primary key porque no se puede repetir.
#bueno, primarykey será la unión de nombre user y el id del usuario concatenados
# esta requests tiene paginación con limite... pero para lo que necesitamos no queremos más de 1 pag por usuario.

In [15]:
#iteramos por la lista de amiguismos para observar todas las keys de cada friend
for fri in req_fri['user']:
    print(fri.keys()) #son los datos de usuario que luego se pueden conseguir, por lo que nos quedmso solo con los "name"

dict_keys(['name', 'url', 'country', 'playlists', 'playcount', 'image', 'registered', 'realname', 'subscriber', 'bootstrap', 'type'])
dict_keys(['name', 'url', 'country', 'playlists', 'playcount', 'image', 'registered', 'realname', 'subscriber', 'bootstrap', 'type'])
dict_keys(['name', 'url', 'country', 'playlists', 'playcount', 'image', 'registered', 'realname', 'subscriber', 'bootstrap', 'type'])
dict_keys(['name', 'url', 'country', 'playlists', 'playcount', 'image', 'registered', 'realname', 'subscriber', 'bootstrap', 'type'])
dict_keys(['name', 'url', 'country', 'playlists', 'playcount', 'image', 'registered', 'realname', 'subscriber', 'bootstrap', 'type'])
dict_keys(['name', 'url', 'country', 'playlists', 'playcount', 'image', 'registered', 'realname', 'subscriber', 'bootstrap', 'type'])
dict_keys(['name', 'url', 'country', 'playlists', 'playcount', 'image', 'registered', 'realname', 'subscriber', 'bootstrap', 'type'])
dict_keys(['name', 'url', 'country', 'playlists', 'playcount',

In [16]:
lista_amigos=[]
for fri in req_fri['user']:
    lista_amigos.append(fri['name'].lower())

In [17]:
lista_amigos #para conseguir la infor iríamos iterando por esta lista y llamando a la api get.info de user

['lumanet',
 'trickywagon',
 'crisnuka',
 'ghan64',
 'aiko4985',
 'ljulycastro',
 'rubendomfer',
 'mister__moss',
 'taneli1',
 'avekaesar']

In [18]:
# a la hora de insertarr la info de friends en mysql, primero comprobar si el usuario ya existe. (capitán obvio)

## Conseguir los recenttracks

In [19]:
url_track = f'http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={l_user}&api_key={keylast}&limit=1000&format=json'

In [20]:
req_track = requests.get(url_track).json()['recenttracks']

In [21]:
req_track.keys()

dict_keys(['track', '@attr'])

In [22]:
req_track['@attr'] #aquí podemos observar la paginación... 

{'user': 'sinatxester',
 'totalPages': '78',
 'page': '1',
 'total': '77557',
 'perPage': '1000'}

In [23]:
req_track['track'][0] #puede que el primer elemento sea #nowplaying!! cuidaó con esto 
# el nowplaying está en todas las páginas 

{'artist': {'mbid': '', '#text': 'Quique González'},
 'streamable': '0',
 'image': [{'size': 'small',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'medium',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'large',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'extralarge',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png'}],
 'mbid': '',
 'album': {'mbid': '', '#text': 'La Noche Americana'},
 'name': 'Se equivocaban contigo',
 'url': 'https://www.last.fm/music/Quique+Gonz%C3%A1lez/_/Se+equivocaban+contigo',
 'date': {'uts': '1651064362', '#text': '27 Apr 2022, 12:59'}}

In [24]:
req_track['track'][1].keys()

dict_keys(['artist', 'streamable', 'image', 'mbid', 'album', 'name', 'url', 'date'])

In [25]:
req_track['track'][1] #nos vamos a quedar solo con una serie de información
#artist: dic (Ambos)
#mbid: si no hay da igual, es el mbid del tema (musicbrainz)
#album dic (ambos)
#name 
# url
#date: dic. nos quedamos solo con uts (También tenemos hecha ya una función que nos da el uts)

{'artist': {'mbid': '27e948aa-6b38-4da2-acf6-8459964a130a',
  '#text': 'Quique González'},
 'streamable': '0',
 'image': [{'size': 'small',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'medium',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'large',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png'},
  {'size': 'extralarge',
   '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png'}],
 'mbid': '',
 'album': {'mbid': '', '#text': 'La Noche Americana'},
 'name': 'Nunca escaparán',
 'url': 'https://www.last.fm/music/Quique+Gonz%C3%A1lez/_/Nunca+escapar%C3%A1n',
 'date': {'uts': '1651064248', '#text': '27 Apr 2022, 12:57'}}

# conseguir info de artista
- una vez que tenemos el nombre y el mbid podemos ir al endpoint del artista
- también con mbid podemos sacar infor del artista de la api de musicbrainz

### endpoint lastfm

In [49]:
artista = req_track['track'][1]['artist']['#text']
art_mbid = req_track['track'][1]['artist']['mbid']
artista,art_mbid

('Quique González', '27e948aa-6b38-4da2-acf6-8459964a130a')

In [27]:
url_art = f'http://ws.audioscrobbler.com/2.0/?method=artist.getinfo&artist={artista}&api_key={keylast}&format=json'

In [28]:
req_art = requests.get(url_art).json()['artist']

In [29]:
req_art
# las imágenes parece que son todas la estrellita... ¿cómo descargarse la imagen real?
 # para conseguir la imagen hay que recurrir a otras bases de datos como la api de musicbrainz


{'name': 'Quique González',
 'mbid': '27e948aa-6b38-4da2-acf6-8459964a130a',
 'url': 'https://www.last.fm/music/+noredirect/Quique+Gonz%C3%A1lez',
 'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'small'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'medium'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'large'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'extralarge'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': 'mega'},
  {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
   'size': ''}],
 'streamable': '0',
 'ontour': '0',
 'stats': {'listeners': '83319', 'playcount': '2485023'},
 'similar': {'artist': [{'name': 'Ismael Serra

In [30]:
req_art.keys()

dict_keys(['name', 'mbid', 'url', 'image', 'streamable', 'ontour', 'stats', 'similar', 'tags', 'bio'])

In [31]:
# nos vamos a quedar con la sig info
#name, mbid, url, #stats(listeners+playcount)
# puede que nos quedemos también con similar y tags.

### get.tag del artista en cuestión

In [32]:
req_art['tags'] #info artista

{'tag': [{'name': 'cantautor', 'url': 'https://www.last.fm/tag/cantautor'},
  {'name': 'spanish', 'url': 'https://www.last.fm/tag/spanish'},
  {'name': 'singer-songwriter',
   'url': 'https://www.last.fm/tag/singer-songwriter'},
  {'name': 'rock', 'url': 'https://www.last.fm/tag/rock'},
  {'name': 'pop', 'url': 'https://www.last.fm/tag/pop'}]}

In [33]:
url_art_tag = f'http://ws.audioscrobbler.com/2.0/?method=artist.gettoptags&artist={artista}&api_key={keylast}&format=json'

In [34]:
req_art_tag = requests.get(url_art_tag).json()['toptags']

In [35]:
req_art_tag.keys()

dict_keys(['tag', '@attr'])

In [36]:
req_art_tag['@attr']

{'artist': 'Quique González'}

In [37]:
req_art_tag['tag'][0] #lista
#cada elemento de la lista es un diccioanrio con count,name y url de keys

{'count': 100, 'name': 'cantautor', 'url': 'https://www.last.fm/tag/cantautor'}

In [38]:
for arttag in req_art_tag['tag']: 
    print(arttag['count'], arttag['name'])

100 cantautor
91 spanish
91 singer-songwriter
89 rock
54 pop
10 indie
7 seen live
7 folk
7 Spanish Rock
7 Cantautores
7 cancion de autor
7 quique gonzalez
5 classic rock
5 country
5 alternative
5 folk-rock
5 spain
5 español
5 male vocalists
5 spanish pop
5 Madrid
3 jazz
3 Grunge
3 pop rock
3 country-rock
3 britpop
3 latin
3 blues
3 singer
3 Alt-country
3 folk rock
3 latino
3 Alternative Folk
3 Rock en Espanol
3 Pop-Rock
3 latin pop
3 spanish indie pop
3 Espana
3 franz ferdinand
3 Bob Dylan
3 All
3 my music
3 lentas
3 Dylan
3 genial
3 spanish indie
3 mediterranean rock
3 Spanish Pop Rock
3 Bunbury
3 canalla
3 patxungo
3 vorgemerkt
3 acustica
3 to listen for jan
3 que genio
3 la vida tiene otro sabor y espanya es la mejor
3 para conocer
3 spanish rock-pop
3 pablo sciuto
3 bilbao live 2008
3 chanson espagnole
3 Qduique Gonzalez


## Búsqueda de artista. Resultados

In [39]:
url_art_bus = f'http://ws.audioscrobbler.com/2.0/?method=artist.search&artist={artista}&api_key={keylast}&format=json'

In [40]:
req_art_bus = requests.get(url_art_bus).json()['results']

In [41]:
req_art_bus

{'opensearch:Query': {'#text': '',
  'role': 'request',
  'searchTerms': 'Quique González',
  'startPage': '1'},
 'opensearch:totalResults': '380',
 'opensearch:startIndex': '0',
 'opensearch:itemsPerPage': '30',
 'artistmatches': {'artist': [{'name': 'Quique González',
    'listeners': '83319',
    'mbid': '27e948aa-6b38-4da2-acf6-8459964a130a',
    'url': 'https://www.last.fm/music/Quique+Gonz%C3%A1lez',
    'streamable': '0',
    'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'small'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'medium'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'large'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'extralarge'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2

## información del album

In [48]:
album = req_track['track'][1]['album']['#text']

In [50]:
url_alb = f'http://ws.audioscrobbler.com/2.0/?method=album.getinfo&api_key={keylast}&artist={artista}&album={album}&format=json'

In [54]:
req_alb = requests.get(url_alb).json()['album']

In [55]:
req_alb.keys()

dict_keys(['artist', 'mbid', 'tags', 'name', 'image', 'tracks', 'listeners', 'playcount', 'url'])

In [59]:
req_alb['tags']

{'tag': [{'url': 'https://www.last.fm/tag/cantautor', 'name': 'cantautor'},
  {'url': 'https://www.last.fm/tag/singer-songwriter',
   'name': 'singer-songwriter'},
  {'url': 'https://www.last.fm/tag/rock', 'name': 'rock'},
  {'url': 'https://www.last.fm/tag/alternative+rock',
   'name': 'alternative rock'},
  {'url': 'https://www.last.fm/tag/folk', 'name': 'folk'}]}

In [60]:
req_alb['name']

'La Noche Americana'

In [64]:
req_alb['image'][2]['#text'] #si tiene información... es una lista index 2 = large

'https://lastfm.freetls.fastly.net/i/u/174s/cb17b55ab3c94a87adfec7a38cffd7f1.png'

### tags albums

In [65]:
url_alb_tag = f'http://ws.audioscrobbler.com/2.0/?method=album.gettoptags&artist={artista}&album={album}&api_key={keylast}&format=json'

In [68]:
req_alb_tag = requests.get(url_alb_tag).json()['toptags']

In [72]:
req_alb_tag

{'tag': [],
 '@attr': {'artist': 'Quique González', 'album': 'La noche americana'}}

In [69]:
req_alb_tag.keys()

dict_keys(['tag', '@attr'])

In [70]:
req_alb_tag['@attr']

{'artist': 'Quique González', 'album': 'La noche americana'}

In [71]:
req_alb_tag['tag']

[]

In [73]:
# con quique gonzález nofunciona mucho... probaré con el ejemplo que ponene de radiohead

In [74]:
url_tag_alb = f'http://ws.audioscrobbler.com/2.0/?method=album.gettoptags&artist=radiohead&album=the%20bends&api_key={keylast}&format=json'

In [76]:
req_tag_alb = requests.get(url_tag_alb).json()['toptags']

In [78]:
req_tag_alb['@attr']

{'artist': 'Radiohead', 'album': 'The Bends'}

In [82]:
req_tag_alb['tag'] # es una lista
for altag in req_tag_alb['tag']:
    print(altag['count'], altag['name']) # hay que tener cuidado con los tagas porque no son todos géneros.

100 albums I own
52 favorite albums
51 alternative rock
41 radiohead
39 rock
33 alternative
28 90s
23 1995
18 favourite albums
15 indie
12 britpop
10 british
8 indie rock
5 the bends
4 UK
3 classic albums
3 experimental
3 melancholy
3 1001 Albums You Must Hear Before You Die
3 Masterpiece
3 1990s
3 1996
3 favorite album
3 cd collection
2 vinyl
2 Favorite
2 albums I love
2 best albums
2 best album ever
2 brit-rock
2 Radiohead - The Bends
2 classic
2 Progressive rock
2 favorites
2 Awesome
2 alt rock
2 oxford
2 overrated
2 cathartic
2 own
2 Grunge
2 Love
2 acoustic
2 england
2 Reflective
2 genius
2 aggressive
2 earnest
2 somber
2 best
2 poignant
2 brooding
2 albums
2 thom yorke
2 gloomy
2 wistful
2 Britrock
2 theatrical
2 wintry
2 urgent
2 plaintive
2 Angst-Ridden
2 Great album
2 no radiohead radio
2 great albums
2 radiohead the bends
